# **Задача 1**
В задаче 1 необходимо сформировать инвестиционный портфель из 100 компаний, ограничений на число акций одной компании не было, но всего было $10^6$ долларов. Были даны стоимости акций за 100 периодов, необходимо было добиться максимальной средней доходности при риске меньшем или примерно равном 0,2.

Наша команда приняла решение свести данную задачу к QUBO-задаче. Для того, чтобы учесть необходимость минимизации риска при максимизации доходности, нами было принято решение за основную часть Гамильтониана взять матрицу ковариаций акций, а за штрафную часть - квадратичную разность стоимости нашего портфеля и $10^6$. При этом, поскольку QUBO лишь бинаризует итоговый вектор, а акций можно взять сколько угодно, мы перевели их число в двоичные, т.е. 101 кодирует 5 и т.д.
Получили $$H = H^{cov}+H^{pen},$$
$$H^{cov} = \sum_{i,j}Cov(i,j)\sum^N_lp_l2^{l-1}q_{i,l}\sum^N_lp_l2^{l-1}q_{j,l}$$
$$H^{pen} = \sum_i(\sum^N_lp_l2^{l-1}q_{i,l})$$
Для задачи QUBO
$$Q = \chi H^{cov} + \lambda(H^{pen}-K)^2,$$ после чего из второго слагаемого было отброшено $\lambda K^2$ и учтено, что квадраты и перекрестные значения $q_iq_j$ могут принимать только значения (0,1)



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyqiopt as pq

n = 100
K = 100000
lambda_penalty = 100

num_of_dec = 10 #Сколько разрядов мы будем кодировать

df = pd.read_csv('task-1-stocks.csv')
profit = df.pct_change().dropna()
cov = profit.cov() #Посчитали ковариации

P_i = df.iloc[0].to_numpy()
cov_matrix = cov.to_numpy()

cov_ = np.zeros((num_of_dec * 100, num_of_dec * 100))

coef_matr = np.zeros((num_of_dec, num_of_dec))
for i in range(num_of_dec):
  for j in range(num_of_dec):
    coef_matr[i][j] = 2**(num_of_dec-1-i)*2**(num_of_dec-1-j)
#П

for i in range(100):
  temp = np.zeros((num_of_dec, num_of_dec * 100))
  for j in range(100):
    a = cov_matrix[i, j]
    a_matr = a*coef_matr
    temp[:, num_of_dec * j:num_of_dec * j + num_of_dec] = a_matr
  cov_[num_of_dec * i:num_of_dec * i + num_of_dec, :] = temp

coef_list = np.array([2*2**i for i in range(num_of_dec-1,-1,-1)])
KP = np.zeros((num_of_dec*100, num_of_dec*100))
for i in range(100):
  for j in range(num_of_dec):
    KP[num_of_dec * i + j, num_of_dec * i + j] = K * P_i[i] * coef_list[j]

p_vec = np.zeros(num_of_dec*100)

for i in range(100):
  for j in range(num_of_dec):
    p_vec[num_of_dec * i + j] = coef_list[j] / 2 * P_i[i]

P_i_sq = np.zeros((num_of_dec*100, num_of_dec*100))

for i in range(num_of_dec*100):
  for j in range(num_of_dec*100):
    P_i_sq[i, j] = p_vec[i] * p_vec[j]

QUBO = cov_ + lambda_penalty * (P_i_sq - KP)

result = pq.solve(QUBO)
vec = list(result.vector)

num_of_ac = np.zeros(100)
for i in range(100):
  num_of_ac[i] = (coef_list * 0.5) @ vec[num_of_dec * i: num_of_dec * i + num_of_dec]

costs = df.iloc[0].to_numpy()
p_new = costs @ num_of_ac
ri = np.zeros(99)
for i in range(1, 100):
  costs = df.iloc[i].to_numpy()
  p_old = p_new
  p_new = costs @ num_of_ac
  ri[i - 1] = (p_new - p_old) / p_old

ri_mean = ri.mean()
sigma = np.sqrt(100 * np.sum((ri - ri_mean) ** 2) / 99)

print(ri_mean, sigma)

44324.59893012047

In [ ]:
num_of_dec = 10
coef_matr = np.zeros((num_of_dec, num_of_dec))
for i in range(num_of_dec):
  for j in range(num_of_dec):
    coef_matr[i][j] = 2**(num_of_dec-1-i)*2**(num_of_dec-1-j)
coef_matr

array([[2.62144e+05, 1.31072e+05, 6.55360e+04, 3.27680e+04, 1.63840e+04,
        8.19200e+03, 4.09600e+03, 2.04800e+03, 1.02400e+03, 5.12000e+02],
       [1.31072e+05, 6.55360e+04, 3.27680e+04, 1.63840e+04, 8.19200e+03,
        4.09600e+03, 2.04800e+03, 1.02400e+03, 5.12000e+02, 2.56000e+02],
       [6.55360e+04, 3.27680e+04, 1.63840e+04, 8.19200e+03, 4.09600e+03,
        2.04800e+03, 1.02400e+03, 5.12000e+02, 2.56000e+02, 1.28000e+02],
       [3.27680e+04, 1.63840e+04, 8.19200e+03, 4.09600e+03, 2.04800e+03,
        1.02400e+03, 5.12000e+02, 2.56000e+02, 1.28000e+02, 6.40000e+01],
       [1.63840e+04, 8.19200e+03, 4.09600e+03, 2.04800e+03, 1.02400e+03,
        5.12000e+02, 2.56000e+02, 1.28000e+02, 6.40000e+01, 3.20000e+01],
       [8.19200e+03, 4.09600e+03, 2.04800e+03, 1.02400e+03, 5.12000e+02,
        2.56000e+02, 1.28000e+02, 6.40000e+01, 3.20000e+01, 1.60000e+01],
       [4.09600e+03, 2.04800e+03, 1.02400e+03, 5.12000e+02, 2.56000e+02,
        1.28000e+02, 6.40000e+01, 3.20000e+01

In [ ]:
coef_list = np.array([2*2**i for i in range(num_of_dec-1,-1,-1)])
coef_list

array([1024,  512,  256,  128,   64,   32,   16,    8,    4,    2])